In [113]:
import pandas as pd
import h5py
import matplotlib.pyplot as plt
import numpy as np

In [116]:
df =pd.read_csv('precip_ET.csv',delimiter = ',', skiprows=6).iloc[-730:,:]
mean_air_temp_c = 273.15+((df['tmax (deg c)'] + df['tmin (deg c)'])/2.0)
precip_ms = df['prcp (mm/day)']/86400000
incom_shor_rad=df['dayl (s)']/86400*df['srad (W/m^2)']
vap_pres=df['vp (Pa)']
df

year  yday  dayl (s)  prcp (mm/day)  srad (W/m^2)  swe (kg/m^2)  \
3012  2022     1  34589.86          48.28        168.94           0.0   
3013  2022     2  34624.85          18.09        138.89           0.0   
3014  2022     3  34662.71           0.00        123.00           0.0   
3015  2022     4  34703.41           0.00        262.52           0.0   
3016  2022     5  34746.93           0.00        257.56           0.0   
...    ...   ...       ...            ...           ...           ...   
3737  2023   361  34458.63           2.85        114.40           0.0   
3738  2023   362  34478.94           0.00        143.46           0.0   
3739  2023   363  34502.22           0.00         83.25           0.0   
3740  2023   364  34528.46           0.00         97.01           0.0   
3741  2023   365  34557.63           0.00        236.93           0.0   

      tmax (deg c)  tmin (deg c)  vp (Pa)  
3012         23.43         13.15  1511.53  
3013         15.98          8.71  1125.12  
3014          2.88         -1.37   552.34  
3015          6.76         -4.30   444.14  
3016         10.76         -0.11   605.63  
...            ...           ...      ...  
3737         12.89          6.03   936.76  
3738          8.33          2.17   714.09  
3739          4.56          1.34   672.72  
3740          5.52          1.84   697.30  
3741         12.39          1.66   688.59  

[730 rows x 9 columns]

In [117]:
precip=df.groupby('yday')['prcp (mm/day)'].mean()
cyclic_precip_ms = precip/86400000
dayl=df.groupby('yday')['dayl (s)'].mean()
srad=df.groupby('yday')['srad (W/m^2)'].mean()
cyclic_incom_shor_rad=dayl/86400*srad
tmax=273.15+df.groupby('yday')['tmax (deg c)'].mean()
tmin=273.15+df.groupby('yday')['tmin (deg c)'].mean()
cyclic_mean_air_temp_c = (tmax + tmin)/2.0
cyclic_vap_pres=df.groupby('yday')['vp (Pa)'].mean()

In [118]:
steady_precip_ms=np.mean(precip_ms)*0.6

steady_mean_air_temp_c =np.mean(mean_air_temp_c)

steady_incom_shor_rad=np.mean(incom_shor_rad)

steady_vap_pres=np.mean(vap_pres)

In [119]:
df2=pd.read_csv('precipitation2.csv')
mean_air_temp_c2=df2['DailyAverageDryBulbTemperature']+273.15
precip_ms2=df2['DailyPrecipitation']/86400000
vap_pres2=df2['VP']
df3=pd.read_csv('SW_rad2.csv').iloc[:-30,:]
incom_shor_rad2=df3['short_wav_rad']

In [120]:
total_precip_ms=np.hstack(([steady_precip_ms]*730,cyclic_precip_ms.values,cyclic_precip_ms.values,precip_ms.values,precip_ms2.values))

total_mean_air_temp_c=np.hstack(([steady_mean_air_temp_c]*730,cyclic_mean_air_temp_c.values,cyclic_mean_air_temp_c.values,mean_air_temp_c.values,mean_air_temp_c2.values))

total_vap_pres=np.hstack(([steady_vap_pres]*730,cyclic_vap_pres.values,cyclic_vap_pres.values,vap_pres.values,vap_pres2.values))

total_incom_shor_rad=np.hstack(([steady_incom_shor_rad]*730,cyclic_incom_shor_rad.values,cyclic_incom_shor_rad.values,incom_shor_rad.values,incom_shor_rad2.values))

In [125]:
hf = h5py.File('daymet_data.h5', 'w')

In [126]:
hf.create_dataset('time [s]', data=np.arange(0,86400*len(total_precip_ms),86400))
hf.create_dataset('air temperature [K]', data=total_mean_air_temp_c)
hf.create_dataset('incoming shortwave radiation [W m^-2]', data=total_incom_shor_rad)
hf.create_dataset('vapor pressure air [Pa]', data=total_vap_pres)
hf.create_dataset('precipitation rain [m s^-1]', data=np.where(total_mean_air_temp_c >= 273.15, total_precip_ms, 0))
hf.create_dataset('precipitation snow [m SWE s^-1]', data=np.where(total_mean_air_temp_c < 273.15, total_precip_ms, 0))

<HDF5 dataset "precipitation snow [m SWE s^-1]": shape (2434,), type "<f8">

In [135]:
hf.close()

In [132]:
hf = h5py.File('../../data/MODIS_LAI_typical_2005_2023.h5', 'r')
cyclic_LAI=hf['NLCD Developed, Medium Intensity LAI [-]'][:730] #two years cyclic
LAI2=hf['NLCD Developed, Medium Intensity LAI [-]'][:244] # use for 2024 jan to aug
hf = h5py.File('../../data/MODIS_LAI_smoothed_2005_2023.h5', 'r')
LAI=hf['NLCD Developed, Medium Intensity LAI [-]'][-730:]# 2022 Jan to dec 2023
steady_LAI=np.mean(hf['NLCD Developed, Medium Intensity LAI [-]'][:])

In [133]:
hf = h5py.File('LAI_data.h5', 'w')
hf.create_dataset('time [s]', data=np.arange(0,86400*len(total_precip_ms),86400))
hf.create_dataset('NLCD Developed, Medium Intensity LAI [-]', data=np.hstack(([steady_LAI]*730,cyclic_LAI,LAI,LAI2)))
#hf.close()

<HDF5 dataset "NLCD Developed, Medium Intensity LAI [-]": shape (2434,), type "<f8">

In [2]:
###

In [158]:
STW2=np.loadtxt('predicted_STW2.txt')

In [159]:
total_level=np.hstack(([np.mean(STW2)]*1460,STW2[-815:],np.loadtxt('STW2_avg.txt')[:87]+22,np.loadtxt('STW1.csv')))

In [162]:
hf = h5py.File('Water_level.h5', 'w')

hf.create_dataset('time [s]', data=np.arange(0,86400*len(total_level),86400))
hf.create_dataset('Water_level [m]', data=total_level/100)
hf.close()